# Campaign Investment Analysis

This notebook analyzes planned vs actual campaign investment based on provided plan and post-buy datasets.
The goal is to compare daily and cumulative spend and evaluate budget utilization over time.

## Data Loading

Plan and post-buy data are loaded from Azure Blob Storage using HTTP access with SAS token.

In [ ]:
import sys
sys.path.append("../src")

from extract import load_plan_data, load_postbuy_data

plan_df = load_plan_data()
postbuy_df = load_postbuy_data()

## Initial Data Exploration

Basic checks are performed to understand data structure, data types and missing values.

In [ ]:
plan_df.head()

In [ ]:
postbuy_df.head()

In [ ]:
postbuy_df.info()

In [ ]:
postbuy_df.isna().sum()

## Post-buy Data Cleaning and Daily Aggregation

Post-buy data are reduced to relevant columns, cleaned from missing values and aggregated to daily granularity.

In [ ]:
import pandas as pd

# Vyčistíme data v postbuy_df
postbuy_clean = postbuy_df[[
    "Date",
    "MarketingInvestment"
]]

# Převedeme sloupec "Date" na datetime formát
postbuy_clean["Date"] = pd.to_datetime(
    postbuy_clean["Date"],
    format="%d/%m/%Y"
)

# Odstraníme řádky s chybějícími hodnotami v "MarketingInvestment"
postbuy_clean = postbuy_clean.dropna(subset=["MarketingInvestment"])

# Seskupíme podle data a sečteme investice
postbuy_daily = (
    postbuy_clean
    .groupby("Date", as_index=False)
    .sum()
)

postbuy_daily.head()

In [ ]:
postbuy_daily.info()

## Planned Investment Preparation

Daily planned investment is calculated based on client-level impressions, clicks and views.

In [ ]:
# převod Date na datetime (sjednocení s postbuy)
plan_df["Date"] = pd.to_datetime(plan_df["Date"])

# výpočet plánované denní investice dle zadání
plan_df["PlannedInvestment"] = (
    plan_df["Impressions_Client"]
    + plan_df["Clicks_Client"]
    + plan_df["Views_Client"]
)

# vybereme jen relevantní sloupce
plan_daily = plan_df[["Date", "PlannedInvestment"]]

plan_daily.head()

## Planned vs Actual Daily Comparison

Planned and actual daily investments are joined to enable direct comparison.

In [ ]:
# spojíme plánované a skutečné denní investice
daily_comparison = plan_daily.merge(
    postbuy_daily,
    on="Date",
    how="left"
)

# seřadíme podle data
daily_comparison = daily_comparison.sort_values("Date")

daily_comparison["MarketingInvestment"] = (
    daily_comparison["MarketingInvestment"].fillna(0)
)

daily_comparison.head()

In [ ]:
daily_comparison.info()

In [ ]:
import matplotlib.pyplot as plt

# Vykreslíme graf plánovaných vs skutečných denních investic
plt.figure(figsize=(12, 6))

# Vykreslíme plánované investice
plt.plot(
    daily_comparison["Date"],
    daily_comparison["PlannedInvestment"],
    label="Planned Investment"
)

# Vykreslíme skutečné investice
plt.plot(
    daily_comparison["Date"],
    daily_comparison["MarketingInvestment"],
    label="Actual Investment"
)

plt.title("Planned vs Actual Investment per Day")
plt.xlabel("Date")
plt.ylabel("Investment")
plt.legend()
plt.grid(True)

plt.show()

## Cumulative Investment and Budget Utilization

Cumulative view is used to evaluate overall budget consumption over time.

In [ ]:
# kumulované hodnoty
daily_comparison["Planned_Cumulative"] = (
    daily_comparison["PlannedInvestment"].cumsum()
)

daily_comparison["Actual_Cumulative"] = (
    daily_comparison["MarketingInvestment"].cumsum()
)

# převod na miliony pro lepší čitelnost grafu
daily_comparison["Planned_Cumulative_M"] = (
    daily_comparison["Planned_Cumulative"] / 1_000_000
)

daily_comparison["Actual_Cumulative_M"] = (
    daily_comparison["Actual_Cumulative"] / 1_000_000
)

# Vykreslíme graf kumulovaných plánovaných vs skutečných investic
plt.figure(figsize=(12, 6))

plt.plot(
    daily_comparison["Date"],
    daily_comparison["Planned_Cumulative_M"],
    label="Planned Cumulative Investment"
)

plt.plot(
    daily_comparison["Date"],
    daily_comparison["Actual_Cumulative_M"],
    label="Actual Cumulative Investment"
)

plt.title("Cumulative Planned vs Actual Investment")
plt.xlabel("Date")
plt.ylabel("Investment (millions)")
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
# výpočet procenta využití rozpočtu
daily_comparison["Budget_Utilization_%"] = (
    daily_comparison["Actual_Cumulative"]
    / daily_comparison["Planned_Cumulative"]
) * 100

daily_comparison[["Date", "Budget_Utilization_%"]].tail()